<div style="background-color:#f0f0f0; padding:20px; border-radius:12px; max-width:1490px; width:100%; box-sizing:border-box;">

  <h1 style="margin:0; color:#2c3e50; font-size:32px;">001 🏠 Cleaning Setup</h1>

  <p style="margin:5px 0 0 0; font-size:16px; color:#34495e;">
    <strong>Authors:</strong> Cecil Quibranza & Matthew Israel &nbsp;&nbsp;|&nbsp;&nbsp; 
    <strong>Date:</strong> 2025-08-30
  </p>

  <hr style="margin:10px 0; border:none; border-top:2px solid #dcdcdc;">

  <p style="margin:0; font-size:16px; color:#34495e;">
    This is the <strong>first notebook</strong>, focusing on the <strong>cleaning of images</strong>. 
    The goal is to standardize and prepare datasets by removing redundant or irrelevant data, ensuring that only high-quality, non-augmented images are retained. 
    This process minimizes dataset noise and prevents biases during training.
  </p>

  <ol style="margin:10px 0 0 20px; color:#34495e; font-size:16px; list-style-position:inside;">
    <li><strong>Extract:</strong> Unzip the dataset files into their respective folders to make all images accessible for preprocessing.</li>
    <li><strong>Remove augmented/duplicates:</strong> Manually remove rotated/mirrored/heavily filtered images and exact duplicates. 
        Keep only originals to reduce imbalance and overfitting.</li>
    <li><strong>Move cleaned images:</strong> Consolidate remaining images into a single folder structure by class (e.g., disease/condition). 
        This enforces a uniform layout for loading and evaluation.</li>
    <li><strong>Repeat for all sources:</strong> Apply steps 1–3 across every dataset to maintain consistency for downstream training.</li>
  </ol>

  <p style="margin:10px 0 0 0; font-size:15px; color:#7f8c8d;">
    ✅ This establishes a clean baseline so later steps (resize, normalization, controlled augmentation) operate on unbiased data.
  </p>
</div>
